In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib

In [11]:
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
matplotlib.rcParams['axes.unicode_minus']=False

In [12]:
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 3. Base Line

In [13]:
import pickle
with open('raw_data_20180811.pickle','rb') as handle:

    raw_data = pickle.load(handle)

In [14]:
kospi200 = pd.read_excel("C:\\Users\\axasd\\Downloads\\상장법인목록.xlsx")
kospi200 = kospi200[kospi200.columns[:2]]
kospi200["종목코드"] = kospi200["종목코드"].map('{:06d}'.format)

In [15]:
code_dict = {
    i : j for i, j in zip(kospi200["회사명"], kospi200["종목코드"])
}

In [16]:
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(ma_7 = raw_data[i].Close.rolling(window =7).mean())
    raw_data[i] = raw_data[i].assign(ma_3 = raw_data[i].Close.rolling(window = 3).mean())
    raw_data[i] = raw_data[i].assign(ma_5 = raw_data[i].Close.rolling(window = 5).mean())
    raw_data[i] = raw_data[i].assign(ma_10 = raw_data[i].Close.rolling(window = 10).mean())
    


In [18]:
#Y값 계산하는 코드 이거하고 아래 2개
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(ma_close_diff = raw_data[i].Close-raw_data[i].ma_7)

In [20]:
for i in raw_data.keys():
    raw_data[i] = raw_data[i].assign(Y = np.where(raw_data[i]["ma_close_diff"].fillna(0)>=0, 1,0))

In [21]:
for i in raw_data.keys():
    raw_data[i] = raw_data[i].dropna(axis=0)

In [23]:
copy_keys = kospi200["종목코드"]
for i in copy_keys:
    if (len(raw_data[i]) < 252):
        del raw_data[i]

### Train, Test Split

In [24]:
train = {}
train_idx = {}
test = {}
test_idx = {}
for code, df in raw_data.items():
    train[code] = df.iloc[:int(len(df)*0.8)][["Open","ma_3","ma_5", "ma_10"]]
    train_idx[code] = df.iloc[:int(len(df)*0.8)]["Y"]
    test[code] = df.iloc[int(len(df)*0.8):][["Open","ma_3","ma_5", "ma_10"]]
    test_idx[code] = df.iloc[int(len(df)*0.8):]["Y"]

### Scale 조정

In [25]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
li = ["Open","ma_3","ma_5", "ma_10"]
for code in raw_data.keys():
    train[code][li] = std_scaler.fit_transform(train[code][li].values)
    test[code][li] = std_scaler.fit_transform(test[code][li].values)

### model 호출

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

log_clf = LogisticRegression()
DTC_clf = DecisionTreeClassifier()
svm_clf = SVC()

### Predict

In [30]:
from sklearn.metrics import accuracy_score
score_list = pd.DataFrame(columns=('LR','DT','SVM'))

for i in raw_data.keys():
    a=[]
    X_train = train[i].values
    y_train = train_idx[i].values
    X_test = test[i].values
    y_test = test_idx[i].values
    for clf in (log_clf, DTC_clf, svm_clf):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        a.append(accuracy_score(y_test, y_pred))
    score_list.loc[i] = a

In [31]:
score_list.describe()

,LR,DT,SVM
count,197.000000,197.000000,197.000000
mean,0.772219,0.705328,0.732106
std,0.050259,0.041973,0.068172
min,0.371517,0.443902,0.312195
25%,0.761610,0.687307,0.708978
50%,0.780186,0.708978,0.752322
75%,0.795666,0.730650,0.773994
max,0.835913,0.792570,0.832817
